# Hand Sign Recognition - Train Model

This notebook is used to train the model

## Imports

In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from pathlib import Path
import random

## Load Data

In [ ]:
DATA_DIR = Path("..") / "data" / "processed"
IMG_SIZE = 128

def load_data():
    X = []
    y = []
    label_map = {}
    i = 0

    # For each label directory in the data/processed directory
    for label_dir in DATA_DIR.iterdir():
        if not label_dir.is_dir():
            continue

        label = label_dir.name
        label_map[i] = label

        # For each image file in the label directory
        for img_file in label_dir.glob("*.jpg"):
            img = cv2.imread(str(img_file))
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Add the processed image into array with label index
            X.append(img)
            y.append(i)

        i += 1

    # Normalize Pixel values to be between 1 and 0
    X = np.array(X) / 255.0
    y = np.array(y)
    return X, y, label_map

X, y, label_map = load_data()
print(f"Loaded {len(X)} images across {len(label_map)} labels.")

## Train/Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

## Build CNN Model

In [ ]:
def build_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.00001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    return model

model = build_model((IMG_SIZE, IMG_SIZE, 3), num_classes=len(label_map))
model.summary()

## Train Model

In [ ]:
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    verbose=1,
    min_lr=1e-6
)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=12,
    batch_size=16,
    callbacks=[lr_scheduler]
)

## Plot Training History

In [ ]:
plt.figure(figsize=(12, 4))

# Plot Accuracy Over Epochs
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="val")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

# Plot Loss Over Epochs
plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="val")
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.tight_layout()
plt.show()

## Save Model

In [ ]:
MODEL_DIR = Path("..") / "models"
MODEL_DIR.mkdir(exist_ok=True)

model.save(MODEL_DIR / "cnn_hand_sign_model_05.h5")
print("✅ Model saved!")